In [1]:
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import glob
import cv2

import os

#TRAINGING SET:

In [2]:
def getImagesfromWang(no_of_classes):
    insect_images = []
    labels = []
    i = 0
    for insect_dir_path in glob.glob(r"C:\Users\aimla\Desktop\insect\Wang dataset/*"):
        insect_label = insect_dir_path.split("/")[-1]
        
        if no_of_classes == i:
            break
        for image_path in glob.glob(os.path.join(insect_dir_path, "*.jpg")):
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)

            image = cv2.resize(image, (64, 64)) 
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            insect_images.append(image)
            labels.append(insect_label)
        i = i + 1

    insect_images = np.array(insect_images)
    labels = np.array(labels)
    label_to_id_dict = {v: i for i, v in enumerate(np.unique(labels))}
    id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}
    id_to_label_dict


    label_ids = np.array([label_to_id_dict[x] for x in labels])
    insect_images.shape, label_ids.shape, labels.shape
    
    print ("Total Images : " +  str(insect_images.shape[0]))
    return insect_images, label_ids

In [3]:
insects_9_classes, labels_9_classes = getImagesfromWang(no_of_classes=9)
insects_5_classes, labels_5_classes = getImagesfromWang(no_of_classes=5)

Total Images : 1359
Total Images : 755


In [4]:
#SETTING UP THE NEURAL NETWORK
def modelCNN(X_train, X_test, Y_train, Y_test, Y, numClasses):
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    
    model_cnn = Sequential()
    # First convolutional layer, note the specification of shape
    model_cnn.add(Conv2D(32, kernel_size=(3,3),
                     activation='relu',
                     input_shape=(64, 64, 3)))

    #Second layer
    model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
    #model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
    model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

    #Third Layer
    model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
    #model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
    model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

    #Fourth layer
    model_cnn.add(Conv2D(128, (3, 3), activation='relu'))

    model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
    model_cnn.add(MaxPooling2D(pool_size=(2, 2)))


    model_cnn.add(Dropout(0.25))
    model_cnn.add(Flatten())
    model_cnn.add(Dense(128, activation='relu'))
    model_cnn.add(Dropout(0.5))
    model_cnn.add(Dense(numClasses, activation='softmax'))

    opt = keras.optimizers.Adam(lr=0.001)
    model_cnn.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['accuracy'])

   

    model_cnn.fit(X_train, Y_train,
              batch_size=64,
              epochs=50,
              verbose=1,
              validation_data=(X_test, Y_test))

    predict = model_cnn.predict(X_test, batch_size=1)
    y = np.argmax(predict, axis=1)

    
    print('Accuracy Score :',accuracy_score(Y, y))
 
    return accuracy_score(Y, y)  


In [9]:
from sklearn.model_selection import KFold
import keras
from tensorflow.keras.utils import to_categorical
def getKFoldCV(Images, Labels, numClasses):
    # KFold Cross Validation approach
    kf = KFold(n_splits=9,shuffle=True,random_state=1245)
    kf.split(Images)

    # Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
    accuracy_model = []

    # Iterate over each train-test split
    for train_index, test_index in kf.split(Images):
        # Split train-test
        X_train, X_test = Images[train_index], Images[test_index]
        Y_train, Y_test = Labels[train_index], Labels[test_index]
        Y=Y_test

        #Normalize color values to between 0 and 1
        X_train = X_train/255
        X_test = X_test/255

        #Make a flattened version for some of our models
        X_flat_train = X_train.reshape(X_train.shape[0], 64*64*3)
        X_flat_test = X_test.reshape(X_test.shape[0], 64*64*3)

        #One Hot Encode the Output
        Y_train =tf.keras.utils.to_categorical(Y_train,numClasses)
        Y_test = tf.keras.utils.to_categorical(Y_test,numClasses)

        # Train the model
        print('Original Sizes:', X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
        accuracy_model.append(modelCNN(X_train, X_test, Y_train, Y_test, Y, numClasses))
    
    return (accuracy_model) 


     


   

In [10]:
import time
start_time= time.time()
accurary_5Classes = getKFoldCV(insects_5_classes, labels_5_classes, 5)
end_time=time.time()
print(accurary_5Classes)
print("Total time taken {}".format(end_time-start_time)) 

Original Sizes: (671, 64, 64, 3) (84, 64, 64, 3) (671, 5) (84, 5)


D:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
11/11 [==============================] - 34s 802ms/step - loss: 1.5573 - accuracy: 0.2617 - val_loss: 1.2836 - val_accuracy: 0.3929
Epoch 2/50
11/11 [==============================] - 7s 668ms/step - loss: 1.2868 - accuracy: 0.4538 - val_loss: 1.2232 - val_accuracy: 0.4643
Epoch 3/50
11/11 [==============================] - 7s 673ms/step - loss: 1.1507 - accuracy: 0.5130 - val_loss: 1.0843 - val_accuracy: 0.5238
Epoch 4/50
11/11 [==============================] - 7s 670ms/step - loss: 1.0086 - accuracy: 0.6005 - val_loss: 0.9779 - val_accuracy: 0.6190
Epoch 5/50
11/11 [==============================] - 7s 671ms/step - loss: 0.9940 - accuracy: 0.6234 - val_loss: 0.9737 - val_accuracy: 0.6190
Epoch 6/50
11/11 [==============================] - 7s 670ms/step - loss: 0.8229 - accuracy: 0.6641 - val_loss: 0.9084 - val_accuracy: 0.5952
Epoch 7/50
11/11 [==============================] - 7s 671ms/step - loss: 0.7540 - accuracy: 0.7042 - val_loss: 0.7417 - val_accuracy: 0.6905
Epoch

D:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
11/11 [==============================] - 11s 771ms/step - loss: 1.5104 - accuracy: 0.3003 - val_loss: 1.3125 - val_accuracy: 0.4286
Epoch 2/50
11/11 [==============================] - 8s 697ms/step - loss: 1.2741 - accuracy: 0.4356 - val_loss: 1.1044 - val_accuracy: 0.5000
Epoch 3/50
11/11 [==============================] - 7s 681ms/step - loss: 1.0951 - accuracy: 0.5277 - val_loss: 0.8703 - val_accuracy: 0.6190
Epoch 4/50
11/11 [==============================] - 7s 662ms/step - loss: 0.9512 - accuracy: 0.6291 - val_loss: 0.7856 - val_accuracy: 0.7024
Epoch 5/50
11/11 [==============================] - 7s 667ms/step - loss: 0.8597 - accuracy: 0.6599 - val_loss: 0.6640 - val_accuracy: 0.7143
Epoch 6/50
11/11 [==============================] - 7s 673ms/step - loss: 0.7159 - accuracy: 0.7313 - val_loss: 0.6911 - val_accuracy: 0.6905
Epoch 7/50
11/11 [==============================] - 7s 668ms/step - loss: 0.6486 - accuracy: 0.7782 - val_loss: 0.5745 - val_accuracy: 0.7381
Epoch

...

Epoch 1/50
11/11 [==============================] - 13s 984ms/step - loss: 1.4856 - accuracy: 0.3003 - val_loss: 1.2990 - val_accuracy: 0.3214
Epoch 2/50
11/11 [==============================] - 8s 687ms/step - loss: 1.2615 - accuracy: 0.4113 - val_loss: 0.9679 - val_accuracy: 0.5238
Epoch 3/50
11/11 [==============================] - 7s 664ms/step - loss: 1.1219 - accuracy: 0.5237 - val_loss: 0.9005 - val_accuracy: 0.5833
Epoch 4/50
11/11 [==============================] - 7s 669ms/step - loss: 0.9985 - accuracy: 0.6057 - val_loss: 0.8730 - val_accuracy: 0.6071
Epoch 5/50
11/11 [==============================] - 8s 687ms/step - loss: 0.8612 - accuracy: 0.6775 - val_loss: 0.7609 - val_accuracy: 0.6667
Epoch 6/50
11/11 [==============================] - 8s 699ms/step - loss: 0.7911 - accuracy: 0.6847 - val_loss: 0.7880 - val_accuracy: 0.6190
Epoch 7/50
11/11 [==============================] - 8s 723ms/step - loss: 0.7372 - accuracy: 0.6932 - val_loss: 0.7088 - val_accuracy: 0.7143
Epoch

D:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
11/11 [==============================] - 11s 799ms/step - loss: 1.5470 - accuracy: 0.2526 - val_loss: 1.1830 - val_accuracy: 0.4524
Epoch 2/50
11/11 [==============================] - 8s 705ms/step - loss: 1.3910 - accuracy: 0.3833 - val_loss: 1.1161 - val_accuracy: 0.4524
Epoch 3/50
11/11 [==============================] - 7s 675ms/step - loss: 1.1594 - accuracy: 0.5138 - val_loss: 0.9111 - val_accuracy: 0.6667
Epoch 4/50
11/11 [==============================] - 8s 688ms/step - loss: 1.0517 - accuracy: 0.5915 - val_loss: 0.8072 - val_accuracy: 0.6667
Epoch 5/50
11/11 [==============================] - 8s 736ms/step - loss: 0.8745 - accuracy: 0.6581 - val_loss: 0.7457 - val_accuracy: 0.6548
Epoch 6/50
11/11 [==============================] - 8s 686ms/step - loss: 0.8135 - accuracy: 0.6697 - val_loss: 0.6400 - val_accuracy: 0.7381
Epoch 7/50
11/11 [==============================] - 8s 699ms/step - loss: 0.7114 - accuracy: 0.7246 - val_loss: 0.6446 - val_accuracy: 0.7381
Epoch

D:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
11/11 [==============================] - 21s 1s/step - loss: 1.5195 - accuracy: 0.2831 - val_loss: 1.1827 - val_accuracy: 0.5542
Epoch 2/50
11/11 [==============================] - 8s 704ms/step - loss: 1.2835 - accuracy: 0.4100 - val_loss: 1.0574 - val_accuracy: 0.6386
Epoch 3/50
11/11 [==============================] - 14s 1s/step - loss: 1.1105 - accuracy: 0.5276 - val_loss: 0.9779 - val_accuracy: 0.4699
Epoch 4/50
11/11 [==============================] - 8s 772ms/step - loss: 1.0171 - accuracy: 0.5838 - val_loss: 0.7851 - val_accuracy: 0.6627
Epoch 5/50
11/11 [==============================] - 8s 681ms/step - loss: 0.9095 - accuracy: 0.6222 - val_loss: 0.7585 - val_accuracy: 0.7711
Epoch 6/50
11/11 [==============================] - 8s 720ms/step - loss: 0.8357 - accuracy: 0.6607 - val_loss: 0.7236 - val_accuracy: 0.6386
Epoch 7/50
11/11 [==============================] - 8s 725ms/step - loss: 0.6963 - accuracy: 0.7032 - val_loss: 0.6255 - val_accuracy: 0.7711
Epoch 8/50

In [11]:
accurary_5Classes

[0.9761904761904762,
 0.8452380952380952,
 0.9523809523809523,
 0.9047619047619048,
 0.9404761904761905,
 0.8333333333333334,
 0.8928571428571429,
 0.9404761904761905,
 0.9156626506024096]

In [ ]:
#start_time= time.time()
#accurary_9Classes = getKFoldCV(insects_9_classes, labels_9_classes, 9)
#end_time=time.time()
#print(accurary_9Classes)
#print("Total time taken {}".format(end_time-start_time))



In [ ]:
#accurary_9Classes